In [2]:
import sys
from pathlib import Path

reporoot = Path(".").absolute().parent


In [3]:
sys.path.append(str(reporoot))


In [4]:
from c2st.check import c2st


ImportError: /home/steinb95/.cache/pypoetry/virtualenvs/c2st-eabMrJ1O-py3.10/lib64/python3.10/site-packages/scipy/spatial/transform/rotation.cpython-310-x86_64-linux-gnu.so: undefined symbol: _PyGen_Send

In [9]:
help(c2st)

NameError: name 'c2st' is not defined

In [5]:
from __future__ import annotations
import numpy as np
from functools import partial
from numpy.random import default_rng
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier

FIXEDSEED = 1309

ImportError: /home/steinb95/.cache/pypoetry/virtualenvs/c2st-eabMrJ1O-py3.10/lib64/python3.10/site-packages/scipy/linalg/_interpolative.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZGVbN2v_sin

In [16]:
def nn_c2st(
    X: np.ndarray,
    Y: np.ndarray,
    seed: int = FIXEDSEED,
    n_folds: int = 5,
    scoring: str = "accuracy",
    z_score: bool = True,
    noise_scale: Optional[float] = None,
    verbosity: int = 0,
) -> np.ndarray:

    ndim = X.shape[1]
    clf_class = MLPClassifier
    clf_kwargs = {
        "activation": "relu",
        "hidden_layer_sizes": (10 * ndim, 10 * ndim),
        "max_iter": 1000,
        "solver": "adam",
    }

    return c2st(
        X,
        Y,
        seed,
        n_folds,
        scoring,
        z_score,
        noise_scale,
        verbosity,
        clf_class,
        clf_kwargs,
    )

In [10]:
NDIM = 10
max_nsamples = 8096
sample_sizes = [ 2**it for it in range(7,12)]
RNG = default_rng(FIXEDSEED)
print(sample_sizes)

[128, 256, 512, 1024, 2048]


In [11]:
center_normal = partial(RNG.multivariate_normal, mean=np.zeros(NDIM), cov=np.eye(NDIM))
#two times 5 sigma away
far_away = partial(RNG.multivariate_normal, mean=(2*5.)+np.zeros(NDIM), cov=np.eye(NDIM))
#2 sigma away
some_away = partial(RNG.multivariate_normal, mean=(2.)+np.zeros(NDIM), cov=np.eye(NDIM))
#5 sigma away
mid_away = partial(RNG.multivariate_normal, mean=(5.)+np.zeros(NDIM), cov=np.eye(NDIM))
